# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))

dd_px = dd.read_parquet(parquet_files).set_index("ticker")


c:\Users\andri\anaconda3_v2\Lib\site-packages\dask\dataframe\core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        hi_lo_range = x['High'] - x['Low'])
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag_1'] - 1
))

C:\Users\andri\AppData\Local\Temp\ipykernel_8760\2378583350.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [5]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns
npartitions=11242,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [6]:
dd_feat = dd_feat.compute()



In [7]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns
ticker,,,,,,,,,,,,,,
OXY,2001-01-02,11.496737,12.065586,11.466798,11.586555,6.151971,2514244,Energy,Oil & Gas Exploration & Production,2001,NaN,NaN,0.598788,NaN
OXY,2001-01-03,11.496737,11.646434,11.107525,11.317101,6.008902,2321563,Energy,Oil & Gas Exploration & Production,2001,11.586555,6.151971,0.538909,-0.023256
OXY,2001-01-04,11.227282,11.376979,11.062615,11.257222,5.977112,2739490,Energy,Oil & Gas Exploration & Production,2001,11.317101,6.008902,0.314363,-0.005290
OXY,2001-01-05,11.257222,11.766192,11.257222,11.526676,6.120178,3081640,Energy,Oil & Gas Exploration & Production,2001,11.257222,5.977112,0.508970,0.023936
OXY,2001-01-08,11.436858,11.646434,11.137464,11.466798,6.088382,1790699,Energy,Oil & Gas Exploration & Production,2001,11.526676,6.120178,0.508970,-0.005195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2024-06-25,629.169983,630.549988,624.390015,630.469971,630.469971,1271900,Information Technology,Application Software,2024,623.880005,623.880005,6.159973,0.010563
INTU,2024-06-26,628.000000,638.650024,624.489990,637.479980,637.479980,1169600,Information Technology,Application Software,2024,630.469971,630.469971,14.160034,0.011119
INTU,2024-06-27,639.080017,657.159973,635.510010,651.440002,651.440002,1478000,Information Technology,Application Software,2024,637.479980,637.479980,21.649963,0.021899


In [9]:
dd_feat['rolling_average_return'] = dd_feat['returns'].rolling(10).mean()


In [11]:
dd_feat['rolling_average_return']

ticker
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
          ...   
INTU    0.010734
INTU    0.011941
INTU    0.009628
INTU    0.010109
INTU    0.008947
Name: rolling_average_return, Length: 2746831, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
 
 
No, it is possible to calculate the moving average return with Dask DataFrame tools. 

+ Would it have been better to do it in Dask? Why?

It depends. In our case conversion into pandas was favourable because it is generally recommended to use pandas if data fits into the RAM, because pandas can often be faster and easier to use (in comparison with Dask DataFrame).


(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.